In [ ]:
# add in a string matching fuction to get quality from brand and supermarket lines
    # Same for things containing the flavour directly
    # Fundtion to swap nourishing for healthy terms
Reduced Calorie for low fat - insert it anywhere in the string



In [26]:
import pandas as pd
import random
import re

df = pd.read_csv(r"C:\Users\bened\Downloads\synthetic_items.csv")

# Helper function to adjust the case of the replacement to match the original text.
def match_case(replacement, original):
    if original.isupper():
        return replacement.upper()
    elif original.islower():
        return replacement.lower()
    elif original[0].isupper() and original[1:].islower():
        return replacement.capitalize()
    else:
        return replacement

# Synonyms were gathered via internet searches (e.g., Thesaurus.com :contentReference[oaicite:0]{index=0})
# and refined to ensure they exactly match the intended meanings in a food context.

# These phrases will be removed and reinserted in a random position.
swap_position_dict = {
    "nourishing": [
        "nourishing", "nutritious", "wholesome", "healthful", "fortifying", "nurturing", "sustaining"
    ],
    "reduced calorie": [
        "Reduced Calorie", "calorie-reduced", "calorie-lowered", "lower-calorie", "diet-friendly", "calorie-cut", "calorie-smart"
    ],
    "low calorie": [
        "Low Calorie", "low-calorie", "calorie-light", "reduced-calorie", "calorie-conscious", "lean", "light"
    ]
}

# These phrases will be replaced in-place.
direct_replace_dict = {
    "fresh": [
        "fresh", "crisp", "farm-fresh", "just picked", "recently harvested", "newly picked", "vibrant"
    ],
    "spiced": [
        "spicy", "hot", "piquant", "peppery", "fiery", "mouth burning", "spice-infused"
    ],
    "spicy": [  # Added for "spicy" as in hot flavor
        "spicy", "hot", "piquant", "peppery", "fiery", "mouth burning", "spice-infused"
    ],
    "sweet": [
        "sweet", "sugary", "honeyed", "candy-sweet", "mildly sweet", "dessert-like", "saccharine"
    ],
    "salty": [
        "salty", "salted", "briny", "saline", "well-salted", "sea-salty"
    ],
    "bitter": [
        "bitter", "acrid", "harsh", "astringent", "sharp", "pungent"
    ],
    "umami": [
        "umami", "meaty", "brothy", "full-bodied", "earthy"
    ],
    "organic": [
        "organic", "naturally grown", "organically produced", "pesticide-free", "chemical-free", "eco-friendly", "green"
    ],
    "grab & go": [
        "grab & go", "portable", "ready-to-eat", "convenience", "on-the-run", "quick service", "takeaway"
    ],
    "celebrate": [
        "celebrate", "festive", "party", "rejoice", "commemorate", "honor", "jubilee"
    ],
    "event": [
        "event", "occasion", "celebration", "gathering", "festivity", "function", "happening", "affair", "wedding", "funeral"
    ],
    "sour": [
        "sour", "tart", "tangy", "acidic", "vinegary", "sharp", "zesty"
    ],
    "kit": [
        "kit", "cooking package", "easy cook", "cooking kit", "meal kit", "recipe kit", "culinary kit"
    ],
    "healthy": [
        "healthy", "wholesome", "nutritious", "good for you", "balanced"
    ],
    "french": [
        "french", "franco", "gallic", "from France", "french-inspired", "Parisian", "french-style"
    ],
    "british": [
        "british", "UK", "english", "from Britain", "British-style", "London"
    ],
    "essentials": [
        "essentials", "staples", "basics", "necessities", "core items"
    ]
}

# Special dictionary for "Honey" replacements.
honey_swap_dict = {
    "honey": [
        "honey", "sugar syrup", "sweetener", "bee syrup", "nectar"
    ]
}

def replace_phrases(text, l1_category=None):
    # Process phrases that require random repositioning.
    for phrase, synonyms in swap_position_dict.items():
        pattern = re.compile(re.escape(phrase), flags=re.IGNORECASE)
        match = pattern.search(text)
        if match:
            original_case = match.group()
            # Choose a synonym and adjust its case to match the original.
            new_phrase = match_case(random.choice(synonyms), original_case)
            # Remove all occurrences (case insensitive) and clean up extra spaces.
            text = pattern.sub("", text)
            text = " ".join(text.split())
            words = text.split()
            insert_index = random.randint(0, len(words))
            words.insert(insert_index, new_phrase)
            text = " ".join(words)
    
    # Process in-place replacements for all other phrases.
    for phrase, synonyms in direct_replace_dict.items():
        # Use word boundaries to ensure full-word matches.
        pattern = re.compile(r'\b' + re.escape(phrase) + r'\b', flags=re.IGNORECASE)
        text = pattern.sub(lambda m: match_case(random.choice(synonyms), m.group()), text)
    
    # Special handling for "Honey"
    # If the L1 category is "Honey" (case insensitive), do not swap.
    if not (l1_category and l1_category.lower() == "honey"):
        pattern = re.compile(r'\bhoney\b', flags=re.IGNORECASE)
        text = pattern.sub(lambda m: match_case(random.choice(honey_swap_dict["honey"]), m.group()), text)
    
    return text

def count_rows_with_string(df, column, search_str):
    """
    Count rows in DataFrame 'df' where 'column' contains 'search_str' (case-insensitive).
    """
    mask = df[column].str.contains(search_str, case=False, na=False)
    return mask.sum()

# Apply the replacement function to the 'Item Name' column.
# Assumes df has an 'L1' column; if not, it passes None.
df['Item Name'] = df.apply(lambda row: replace_phrases(row['Item Name'], row['L1'] if 'L1' in row else None), axis=1)

df.to_csv(r"G:\My Drive\Wantrepreneurialism\Active\spend-analytics\Tesco Clubcards\5 - Processed Data Files\4) Gathered Data\synthetic_items.csv", index=False)


In [29]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace("_", " "))  # Handle multi-word synonyms
    return list(synonyms)

def get_food_related_synonyms(word):
    """
    Fetch synonyms from WordNet but filter them to ensure they are contextually relevant to food.
    """
    food_related_synonyms = set()
    for synset in wordnet.synsets(word, pos=wordnet.ADJ):  # Only get adjective meanings
        for lemma in synset.lemmas():
            # Check if the synset definition contains food-related words
            definition = synset.definition().lower()
            if any(food_word in definition for food_word in ["food", "flavor", "taste", "cuisine", "spice"]):
                food_related_synonyms.add(lemma.name().replace("_", " "))

    return list(food_related_synonyms)

# Test on food-related words
test_words = ["spicy", "sweet", "sour", "bitter", "salty", "organic", "fresh"]

# Get synonyms for each test word
food_synonyms = {word: get_food_related_synonyms(word) for word in test_words}
food_synonyms



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bened\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


{'spicy': ['zesty', 'savory', 'spicy', 'piquant', 'hot', 'savoury'],
 'sweet': ['sweet'],
 'sour': ['sour'],
 'bitter': ['bitter'],
 'salty': ['salty'],
 'organic': ['organic'],
 'fresh': []}

In [34]:
food_related_synonyms = set()
for synset in wordnet.synsets("apple", pos=wordnet.ADJ):  # Only get adjective meanings
    for lemma in synset.lemmas():
        # Check if the synset definition contains food-related words
        definition = synset.definition().lower()
        if any(food_word in definition for food_word in ["food", "flavor", "taste", "cuisine", "spice"]):
            food_related_synonyms.add(lemma.name().replace("_", " "))

In [39]:
wordnet.synsets("better", pos=wordnet.MORPHOLOGICAL_SUBSTITUTIONS)

[Synset('better.n.01'),
 Synset('bettor.n.01'),
 Synset('better.n.03'),
 Synset('better.n.04'),
 Synset('better.v.01'),
 Synset('better.v.02'),
 Synset('better.v.03'),
 Synset('better.a.01'),
 Synset('better.a.02'),
 Synset('better.s.03'),
 Synset('better.s.04'),
 Synset('good.a.01'),
 Synset('full.s.06'),
 Synset('good.a.03'),
 Synset('estimable.s.02'),
 Synset('beneficial.s.01'),
 Synset('good.s.06'),
 Synset('good.s.07'),
 Synset('adept.s.01'),
 Synset('good.s.09'),
 Synset('dear.s.02'),
 Synset('dependable.s.04'),
 Synset('good.s.12'),
 Synset('good.s.13'),
 Synset('effective.s.04'),
 Synset('good.s.15'),
 Synset('good.s.16'),
 Synset('good.s.17'),
 Synset('good.s.18'),
 Synset('good.s.19'),
 Synset('good.s.20'),
 Synset('good.s.21'),
 Synset('well.a.01'),
 Synset('good.s.13'),
 Synset('well.s.03'),
 Synset('better.r.01'),
 Synset('better.r.02'),
 Synset('well.r.01'),
 Synset('well.r.02'),
 Synset('well.r.03'),
 Synset('well.r.04'),
 Synset('well.r.05'),
 Synset('well.r.06'),
 Syns